<a href="https://colab.research.google.com/github/jrakhshanda/Text-Mining/blob/main/tweet_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweet sentiment extraction - SpaCy
2020-10-24

# Prep

In [4]:
import os

import numpy as np
import pandas as pd
from sklearn import model_selection

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
# drop na
df_train.dropna(axis = 0, how ='any',inplace=True)
train1, test = model_selection.train_test_split(train,test_size=0.2)
train, valid = model_selection.train_test_split(train1,test_size=0.2)

# Prep train set

In [10]:
# Add tokes and counts.
train['text_tokes']   = train.text.str.split()
train['select_tokes'] = train.selected_text.str.split()
train['text_tokes_cnt'] = train.text_tokes.str.len()
train['select_tokes_cnt'] = train.select_tokes.str.len()
train.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,textID,text,selected_text,sentiment,text_tokes,select_tokes,text_tokes_cnt,select_tokes_cnt
10687,5d94fa6526,I love my Boogah! Can anyone contain Lebron Ja...,I love my Boogah! Can anyone contain Lebron Ja...,neutral,"[I, love, my, Boogah!, Can, anyone, contain, L...","[I, love, my, Boogah!, Can, anyone, contain, L...",13,13
11261,7dec86f00e,"lolzz @ public script, wait - n- watch","lolzz @ public script, wait - n- watch",neutral,"[lolzz, @, public, script,, wait, -, n-, watch]","[lolzz, @, public, script,, wait, -, n-, watch]",8,8
2776,1ba776466c,how long did it take you to learn the song,how long did it take you to learn the song,neutral,"[how, long, did, it, take, you, to, learn, the...","[how, long, did, it, take, you, to, learn, the...",10,10
21457,369ac3522c,_archie Haha it`s ok. Glad everything`s good. ...,Glad everything`s good.,positive,"[_archie, Haha, it`s, ok., Glad, everything`s,...","[Glad, everything`s, good.]",10,3
7457,6b4b918764,to my pretty lady HAPPY MOTHER`S DAY!!! She`s...,HAPPY,positive,"[to, my, pretty, lady, HAPPY, MOTHER`S, DAY!!!...",[HAPPY],14,1


In [11]:
# remove text=2 and neutrals as = self.
train = train[~(train.text_tokes_cnt<=2)]
train = train[(train.sentiment!='neutral')]
display(len(train))
display(train.sentiment.value_counts())
train.sample(5)

4270

positive    2259
negative    2011
Name: sentiment, dtype: int64

,textID,text,selected_text,sentiment,text_tokes,select_tokes,text_tokes_cnt,select_tokes_cnt
3720,4034294a60,#followfriday follow these ppl ... they ar...,l ... they are interesting .. doesnt tweet mu...,positive,"[#followfriday, follow, these, ppl, ..., they,...","[l, ..., they, are, interesting, .., doesnt, t...",13,10
24945,74dfd381f2,Getting a cleaning on tues then getting a toot...,ugh.,negative,"[Getting, a, cleaning, on, tues, then, getting...",[ugh.],23,1
7846,9c3beb8b96,Thank you! I hope you have a great rest of yo...,Thank,positive,"[Thank, you!, I, hope, you, have, a, great, re...",[Thank],19,1
19396,d8cfcd20a1,I dont think my twitpic is working,I dont think my twitpic is working,negative,"[I, dont, think, my, twitpic, is, working]","[I, dont, think, my, twitpic, is, working]",7,7
5581,31155fb47b,_Songs I know Especially since the summer is ...,such a boring TV,negative,"[_Songs, I, know, Especially, since, the, summ...","[such, a, boring, TV]",16,4


# Prep validation set - DON'T DO THIS
* DON'T DO THIS - Just rely on model trianing metric.
* Extract 10% of pos and neg into df_valid and remove these from train.
* "A validation dataset is a sample of data held back from training your model that is used to give an estimate of model skill while tuning model’s hyperparameters." - https://machinelearningmastery.com/difference-test-validation-datasets/

In [ ]:
# Hold out set. Ten percent of total train split evenly between sentiment.
# num_rows_to_use = int((len(train) * .10)/2) 
# df_valid_pos = train[train.sentiment=='positive'].sample(num_rows_to_use)
# df_valid_neg = train[train.sentiment=='negative'].sample(num_rows_to_use)
# df_valid = pd.concat([df_valid_pos, df_valid_neg])
# display(len(df_valid))
# display(df_valid.sentiment.value_counts())
# df_valid.sample(5)

In [ ]:
# Drop validation set from training set.
# display(f"Train before={len(train)}")
# train.drop(df_valid.index, inplace=True, axis=0)
# display(f"Train after={len(train)}") 

# New EDA
* Non-neutral sent where input and output are same lengths
* Seem problematic.
* Follow up on later

In [12]:
# Input & output the same.
# Seem problematic to me.
df_sames = train[train.text_tokes_cnt==train.select_tokes_cnt]
display(len(df_sames))
display(df_sames.sentiment.value_counts())
pd.options.display.max_colwidth = 1000
df_sames.sample(5)

602

negative    316
positive    286
Name: sentiment, dtype: int64

,textID,text,selected_text,sentiment,text_tokes,select_tokes,text_tokes_cnt,select_tokes_cnt
26915,983dab0b00,it`s my fave!,it`s my fave!,positive,"[it`s, my, fave!]","[it`s, my, fave!]",3,3
24665,2ddebbe71b,"ate ****, I refuse to bless her cuz she already clingy","ate ****, I refuse to bless her cuz she already clingy",negative,"[ate, ****,, I, refuse, to, bless, her, cuz, she, already, clingy]","[ate, ****,, I, refuse, to, bless, her, cuz, she, already, clingy]",11,11
8779,9bcd32d254,It`s depressing I`m stuck inside all day,It`s depressing I`m stuck inside all day,negative,"[It`s, depressing, I`m, stuck, inside, all, day]","[It`s, depressing, I`m, stuck, inside, all, day]",7,7
12400,daa69a6eb3,Norwood house party haaaaaa... Yaaaaaaay (smiles),Norwood house party haaaaaa... Yaaaaaaay (smiles),positive,"[Norwood, house, party, haaaaaa..., Yaaaaaaay, (smiles)]","[Norwood, house, party, haaaaaa..., Yaaaaaaay, (smiles)]",6,6
7866,029cb0254a,Have fun! Wish your mums a happy mother`s day for us!,Have fun! Wish your mums a happy mother`s day for us!,positive,"[Have, fun!, Wish, your, mums, a, happy, mother`s, day, for, us!]","[Have, fun!, Wish, your, mums, a, happy, mother`s, day, for, us!]",11,11


# Start on the task

## Imports

In [13]:
# Spacy model building related.
import spacy
from tqdm import tqdm
import random
from spacy.util import minibatch, compounding

import warnings
warnings.filterwarnings("ignore")

## model train - helpers

In [14]:
def save_model(output_dir, nlp, new_model_name):
    ''' This Function Saves model to 
    given output directory'''
    
    output_dir = f'../working/{output_dir}'
    if output_dir is not None:        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [15]:
def get_model_out_path(sentiment):
    '''
    Returns Model output path
    '''
    model_out_path = None
    if sentiment == 'positive':
        model_out_path = 'models/model_pos'
    elif sentiment == 'negative':
        model_out_path = 'models/model_neg'
    return model_out_path

In [16]:
def get_training_data(sentiment, df_input):
    '''
    Returns Training data in the format needed to train spacy NER
    ID start and end point of the 'selected' text in the text 
    and used as your string entity info for spacy.
    '''
    SENTIMENT = ['negative', 'positive']
    if sentiment not in SENTIMENT:
        raise ValueError(f"{sentiment} not in {SENTIMENT})")
    train_data = []
    for index, row in df_input.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
    return train_data

In [17]:
# pass model = nlp if you want to train on top of existing model 

def train(train_data, output_dir, n_iter=20, model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    ""
    # Uses given model or instantiates a blank model.
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()

        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts,  # batch of texts
                            annotations,  # batch of annotations
                            drop=0.5,   # dropout - make it harder to memorise data
                            losses=losses, 
                            )
            print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')

In [18]:
def train(train_data, output_dir, n_iter=20, model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    ""
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()


        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts,  # batch of texts
                            annotations,  # batch of annotations
                            drop=0.5,   # dropout - make it harder to memorise data
                            losses=losses, 
                            )
            print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')

In [19]:
def run_train(n_iter=20):
    """ Convenience so can comment out if not don't need to regenerat models. """
    for sentiment in ['positive', 'negative']:
        model_path = get_model_out_path(sentiment)
        train_data = get_training_data(sentiment, train)
        train(train_data, model_path, n_iter=n_iter)

## Build NER models

In [20]:
%%time
run_train(n_iter=5)

AttributeError: ignored

## Testing

In [ ]:
MODELS_BASE_PATH = 'models/'
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

def pred_set(df_set):
    """ Run NER models on data. """
    df_pred = df_set.copy()
    
    selected_texts = []

    print("Loading Models  from ", MODELS_BASE_PATH)
    model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')
    model_neg = spacy.load(MODELS_BASE_PATH + 'model_neg')

    for index, row in df_pred.iterrows():
        text = row.text
        output_str = ""
        if row.sentiment == 'neutral' or len(text.split()) <= 2:
            selected_texts.append(text)
        elif row.sentiment == 'positive':
            selected_texts.append(predict_entities(text, model_pos))
        else:
            selected_texts.append(predict_entities(text, model_neg))

    df_pred['predicted_text'] = selected_texts

    return df_pred

# One-off testing

In [ ]:
model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')

In [ ]:
predict_entities("I love this model, it's great. I can't wait to lean more about training spacy models", model_pos)

## Traning set testing.
* need to use jackard as custom metric during trianing so can skip this

In [ ]:
# Metric.
def jaccard(compare_strings): 
    str1, str2 = compare_strings
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# Predict on validation set - add to df.
df_pred = pred_set(train)

In [ ]:
# Reminder that neutral are missing from this.
df_pred.sentiment.value_counts()

In [ ]:
# Calculate jaccard.
# Do it here then move to above func - probably.
df_pred['jaccard'] = df_pred[['selected_text','predicted_text']].values.tolist()
df_pred['jaccard'] = df_pred.jaccard.apply(jaccard)

In [ ]:
# Not much difference in accuracy per sentiment type
print(df_pred.jaccard.mean())
print(df_pred[df_pred.sentiment=='positive'].jaccard.mean())
print(df_pred[df_pred.sentiment=='negative'].jaccard.mean())

In [ ]:
display(df_pred[['text','selected_text', 'sentiment', 'predicted_text', 'jaccard']].sample(20))

# Leftovers - Ignore this bit

In [ ]:
# Run NER on full test set.
selected_texts = []
MODELS_BASE_PATH = 'models/'

if MODELS_BASE_PATH is not None:
    print("Loading Models  from ", MODELS_BASE_PATH)
    model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')
    model_neg = spacy.load(MODELS_BASE_PATH + 'model_neg')
        
    for index, row in df_test.iterrows():
        text = row.text
        output_str = ""
        if row.sentiment == 'neutral' or len(text.split()) <= 2:
            selected_texts.append(text)
        elif row.sentiment == 'positive':
            selected_texts.append(predict_entities(text, model_pos))
        else:
            selected_texts.append(predict_entities(text, model_neg))
        
df_test['selected_text'] = selected_texts

# Submission prep
* You must have exact same number of rows as sample submission
* Internet must be turned off.
* After saving and running you will need to go to the output notebook and tell it which file to submit.

In [ ]:
print(len(df_test))
print(df_submission.describe())

In [ ]:
df_submission.info()

In [ ]:
df_submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

df_submission = df_test[['textID', 'selected_text']]
print(len(df_submission))
display(df_submission.head(10))

In [ ]:
os.chdir("/kaggle/working/")

In [ ]:
df_submission.to_csv("submission.csv", index=False)

In [ ]:
print(len(df_submission))
print(df_submission.describe())
print(df_submission.info())
df_submission.head(10)

In [ ]:
df_submission

[Refrence code](https://www.kaggle.com/dplutcho/tweet-sent-ext-ner)